<a href="https://colab.research.google.com/github/deepfake-detection-model-project/Deepfake-Detection-Project/blob/jiwon-dev/notebooks/03_augmentation_exp/convnext_different_lr_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install timm # 현재 코랩에 이미 설치되어 있어서 주석처리 함

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
from tqdm import tqdm
import pandas as pd
from google.colab import drive
import os
from collections import defaultdict
import gc
import albumentations as A

In [ ]:
# 1. 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
import sys
# src 폴더를 인식할 수 있도록 경로 추가 (현재 경로가 /content라면)
sys.path.append('/content/drive/MyDrive/HECTO')

from src.utils import set_seed, log_to_csv
from src.dataset import DeepfakeDataset, seed_worker
from src.trainer import train_one_epoch, validate_video_level, save_checkpoint

In [ ]:
# 로컬에서 압축 해제 (네트워크 지연 없음)
!unzip -q /content/drive/MyDrive/HECTO/Dataset/final_files/Celeb_frames_pp_final.zip -d /content/Celeb_Extract
!unzip -q /content/drive/MyDrive/HECTO/Dataset/final_files/DF_Frames_Final.zip -d /content/DF_Extract
!unzip -q /content/drive/MyDrive/HECTO/Dataset/final_files/FF_Frames_Final.zip -d /content/FF++_Extract

In [ ]:
# 2. 체크포인트 저장 경로 설정 (본인의 드라이브 경로에 맞게 수정 가능)
CHECKPOINT_DIR = '/content/drive/MyDrive/HECTO/checkpoints/04_convnext_different_lr_augmentation_only_ff_celeb'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# 3. 데이터셋 경로
CSV_PATH = '/content/drive/MyDrive/HECTO/Dataset/CSV/total_final.csv'

print(f"✅ 체크포인트 저장 경로: {CHECKPOINT_DIR}")

In [ ]:
# 시드 설정
EXPERIMENT_SEED = 42
set_seed(EXPERIMENT_SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ 시드 {EXPERIMENT_SEED} 고정 완료 및 {device} 준비")

In [ ]:
g = torch.Generator()
g.manual_seed(EXPERIMENT_SEED)
IMG_SIZE = 224

# Albumentations와 Torchvision을 연결하기 위한 래퍼 클래스
class AlbumentationsWrapper:
    def __init__(self, transform):
        self.transform = transform
    def __call__(self, img):
        img_np = np.array(img)
        augmented = self.transform(image=img_np)
        return Image.fromarray(augmented['image'])

# --- 1. DF용 (약한 증강) ---
alb_aug_df_settings = A.Compose([
    A.ImageCompression(quality_lower=70, quality_upper=95, p=0.15),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.1),
])

train_transform_df = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    AlbumentationsWrapper(alb_aug_df_settings),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 1.2))], p=0.2), # 강도 낮춤
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15, hue=0.05), # 강도 낮춤
    transforms.CenterCrop((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])

# --- 2. Celeb/FF++용 (기본 증강) ---
alb_aug_celeb_ff_settings = A.Compose([
    A.ImageCompression(quality_lower=50, quality_upper=90, p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
    A.ISONoise(p=0.1)
])

train_transform_others = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    AlbumentationsWrapper(alb_aug_celeb_ff_settings),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))], p=0.3),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.CenterCrop((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])

# 검증용은 동일하게 유지
val_transform = transforms.Compose([
    transforms.CenterCrop((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
class DeepfakeDataset(Dataset):
    def __init__(self, csv_path, split='train', transform_df=None, transform_others=None):
        df = pd.read_csv(csv_path)
        self.data = df[df['split'] == split].reset_index(drop=True)
        # 학습 시에는 두 종류의 transform을 사용, 검증 시에는 하나만 사용하도록 설정
        self.transform_df = transform_df
        self.transform_others = transform_others

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.loc[idx, 'frame_path']
        label = self.data.loc[idx, 'label']
        video_id = self.data.loc[idx, 'video_id']

        image = Image.open(img_path).convert('RGB')

        # 경로에 'DF_Extract'가 포함되어 있는지 확인하여 증강 선택
        if 'DF_Extract' in img_path:
            if self.transform_df:
                image = self.transform_df(image)
        else:
            if self.transform_others:
                image = self.transform_others(image)

        return image, torch.tensor(label, dtype=torch.float32), video_id

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.val_acc_max = 0

    def __call__(self, val_acc):
        score = val_acc

        if self.best_score is None:
            self.best_score = score
            self.val_acc_max = val_acc
        # [수정] 정확도가 개선되지 않은 경우 (score가 best + delta보다 작을 때)
        elif score <= self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        # [수정] 정확도가 확실히 개선된 경우
        else:
            self.best_score = score
            self.val_acc_max = val_acc
            self.counter = 0

def load_checkpoint(model_name, model, optimizer, save_dir=CHECKPOINT_DIR):
    filename = os.path.join(save_dir, f"{model_name}_checkpoint.pth.tar")
    if os.path.isfile(filename):
        print(f"🔄 체크포인트 불러오는 중: {filename}")
        # map_location 추가하여 장치 불일치 방지
        checkpoint = torch.load(filename, map_location=device)

        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        best_acc = checkpoint['best_acc']
        es_state = checkpoint.get('es_state', None)

        # 만약 es_state에 best_score 정보가 없다면 best_acc로 강제 동기화 준비
        if es_state is not None:
            if 'best_score' not in es_state or es_state['best_score'] is None:
                es_state['best_score'] = best_acc

        return start_epoch, best_acc, es_state
    else:
        print("🆕 이전 체크포인트 없음. 처음부터 학습을 시작합니다.")
        return 0, 0, None

In [ ]:
# 1. 원본 CSV 로드
df_orig = pd.read_csv(CSV_PATH)

# 2. 필터링 로직 적용
# 'level_4', 'level_5', 'mix_2'가 경로에 포함된 행은 제외합니다.
# (만약 DF의 level_1, level_2도 있다면, 'level_3'만 명시적으로 남기는 방식이 더 안전할 수 있습니다.)
exclude_list = ['level_3', 'level_4', 'level_5', 'mix_2']

# 문자열을 포함하지 않는(~) 행들만 선택
df_filtered = df_orig[~df_orig['frame_path'].str.contains('|'.join(exclude_list))]

# (추가 권장) 만약 DF 데이터셋 중 level_3만 딱 남기고 싶다면 아래 주석을 해제하세요.
# df_filtered = df_orig[~(df_orig['frame_path'].str.contains('DF_Extract') &
#                        ~df_orig['frame_path'].str.contains('level_3'))]

# 3. 필터링된 결과 저장
FILTERED_CSV_PATH = CSV_PATH.replace('.csv', '_filtered.csv')
df_filtered.to_csv(FILTERED_CSV_PATH, index=False)

# 4. 데이터셋 경로를 필터링된 CSV로 교체
CSV_PATH = FILTERED_CSV_PATH

print(f"✅ 필터링 완료!")
print(f"전체 데이터: {len(df_orig)} -> 필터링 후: {len(df_filtered)}")
print(f"제외된 데이터 수: {len(df_orig) - len(df_filtered)}")

In [ ]:
# 실험 대상 모델
model_names = ['convnext_tiny']

# 하이퍼파라미터
MAX_EPOCHS = 20  # 얼리스탑이 있으므로 넉넉히 잡습니다.
BATCH_SIZE = 32

# 차등 학습률 설정
BACKBONE_LR = 1e-5  # 기존 지식을 보존하기 위해 낮게 설정
HEAD_LR = 1e-4      # 새로운 태스크에 적응하기 위해 10배 높게 설정

# 결과를 저장할 리스트
results_summary = []

# 데이터 로더 준비
# 학습 데이터셋에는 DF용과 기타용 증강을 각각 전달
train_dataset = DeepfakeDataset(CSV_PATH, split='train',
                                transform_df=train_transform_df,
                                transform_others=train_transform_others)

# 검증 데이터셋은 구분할 필요가 없으므로 둘 다 동일한 val_transform 전달
val_dataset = DeepfakeDataset(CSV_PATH, split='val',
                              transform_df=val_transform,
                              transform_others=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2, worker_init_fn=seed_worker, generator=g)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [ ]:
import os
import pandas as pd
import torch
import timm
import gc
from torch import nn, optim

# 결과 저장용 리스트
results_summary = []

for m_name in model_names:
    print(f"\n" + "="*50)
    print(f"🚀 Experiment Target: {m_name}")
    print("="*50)

    # 1. 시드 고정 (재현성 확보)
    set_seed(EXPERIMENT_SEED)

    # 2. 모델 및 기본 객체 초기화
    model = timm.create_model(m_name, pretrained=True, num_classes=1).to(device)
    criterion = nn.BCEWithLogitsLoss()

    optimizer = optim.AdamW([
        {'params': model.stem.parameters(), 'lr': BACKBONE_LR},
        {'params': model.stages.parameters(), 'lr': BACKBONE_LR},
        {'params': model.head.parameters(), 'lr': HEAD_LR}
    ], weight_decay=0.05)

    # 얼리스탑 객체 생성 (기본값으로 시작)
    early_stopping = EarlyStopping(patience=5, verbose=True)

    # 3. [핵심 수정] 체크포인트 로드 (학습 재개 여부 판단의 근거)
    start_epoch, best_acc, es_state = load_checkpoint(m_name, model, optimizer, CHECKPOINT_DIR)

    if es_state is not None:
        # 1. 저장된 상태 전체 복구 (counter, early_stop 등)
        early_stopping.__dict__.update(es_state)

        # 2. [핵심] 만약 best_score가 비어있다면 불러온 best_acc로 채워줌
        if early_stopping.best_score is None:
            early_stopping.best_score = best_acc

        print(f"🔄 {m_name} 상태 복구: Epoch {start_epoch}, Best Acc {best_acc:.4f}, Counter {early_stopping.counter}")

    # 5. [핵심 수정] 학습 완료 여부 판별 로직
    # 조건 A: 이미 얼리스탑이 발생했는가?
    # 조건 B: 이미 설정한 최대 에포크(MAX_EPOCHS)까지 완주했는가?
    if early_stopping.early_stop or start_epoch >= MAX_EPOCHS:
        reason = "Early Stopping 완료" if early_stopping.early_stop else "최대 에포크 완주"
        print(f"✅ {m_name}은(는) 이미 학습이 종료된 상태입니다. ({reason})")
        results_summary.append({'Model': m_name, 'Best Video Acc': best_acc})

        # 메모리 정리 후 다음 모델로 스킵
        del model, optimizer
        torch.cuda.empty_cache()
        gc.collect()
        continue

    # 6. 학습 루프 (start_epoch부터 재개)
    print(f"▶️ {m_name} 학습을 {start_epoch+1} 에포크부터 재개합니다.")

    for epoch in range(start_epoch, MAX_EPOCHS):
        print(f"\n[Epoch {epoch+1}/{MAX_EPOCHS}]")

        # 학습 및 검증
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, video_acc = validate_video_level(model, val_loader, criterion, device)

        print(f"   Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"   Val Loss:   {val_loss:.4f} | Video-level Val Acc: {video_acc:.4f}")

        # 에포크별 로그 기록
        log_to_csv(m_name, epoch+1, train_loss, train_acc, val_loss, video_acc, f"{CHECKPOINT_DIR}/{m_name}_history.csv")

        # 베스트 모델 갱신 확인
        is_best = video_acc > best_acc
        if is_best:
            best_acc = video_acc
            print(f"✨ Best Model Updated! (Acc: {best_acc:.4f})")

        # 얼리스탑 판정 (카운터 업데이트)
        early_stopping(video_acc)

        # 체크포인트 저장 (갱신된 es_state 포함)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc': best_acc,
            'es_state': early_stopping.__dict__,  # 갱신된 얼리스탑 상태 저장
        }, is_best, m_name, CHECKPOINT_DIR)

        if early_stopping.early_stop:
            print(f"🛑 {m_name} Early Stopping triggered at epoch {epoch+1}!")
            break

    results_summary.append({'Model': m_name, 'Best Video Acc': best_acc})

    # 메모리 정리
    del model, optimizer
    torch.cuda.empty_cache()
    gc.collect()

# 7. 최종 결과 보고서
print("\n" + "🏆" * 5 + " Final Comparison Report " + "🏆" * 5)
if results_summary:
    df_res = pd.DataFrame(results_summary)
    # 중복 모델명이 있을 경우 최고 점수만 남김 (재개 로직 대비)
    df_res = df_res.sort_values(by=['Model', 'Best Video Acc'], ascending=[True, False])
    df_res = df_res.drop_duplicates('Model').sort_values(by='Best Video Acc', ascending=False).reset_index(drop=True)
    print(df_res)
    print(f"\n🥇 최적의 백본 모델: {df_res.iloc[0]['Model']} ({df_res.iloc[0]['Best Video Acc']:.4f})")
else:
    print("기록된 실험 결과가 없습니다.")

In [ ]:
# --- 마지막 셀: 학습 완료된 모델의 Video-level AUC 측정 ---

from sklearn.metrics import roc_auc_score
import torch.nn.functional as F
import numpy as np
from collections import defaultdict
from tqdm import tqdm

def get_video_level_auc(model, val_loader, device):
    """
    학습된 모델을 사용하여 비디오 레벨의 AUC를 계산합니다.
    데이터로더가 리스트 [image, label, video_id]를 반환하는 구조에 맞춰 수정되었습니다.
    """
    model.eval()
    video_scores = defaultdict(list)
    video_labels = {}

    print("🔍 AUC 계산을 위한 검증 데이터 추론 중...")
    with torch.no_grad():
        for batch in tqdm(val_loader):
            # 에러 해결: 리스트 인덱싱으로 데이터 추출
            # batch[0]: 이미지, batch[1]: 라벨, batch[2]: 비디오 ID
            images = batch[0].to(device)
            labels = batch[1].to(device)
            video_ids = batch[2]

            outputs = model(images).squeeze()

            # Sigmoid를 적용하여 확률값으로 변환
            probs = torch.sigmoid(outputs).cpu().numpy()

            # 단일 값(Batch Size 1)인 경우 배열로 형태 유지
            if probs.ndim == 0:
                probs = np.array([probs])

            labels = labels.cpu().numpy()

            for i, v_id in enumerate(video_ids):
                video_scores[v_id].append(probs[i])
                if v_id not in video_labels:
                    video_labels[v_id] = labels[i]

    # 비디오별 프레임 확률값의 평균 계산
    all_video_probs = []
    all_video_labels = []

    for v_id in video_scores.keys():
        avg_prob = np.mean(video_scores[v_id])
        all_video_probs.append(avg_prob)
        all_video_labels.append(video_labels[v_id])

    # 최종 AUC 계산
    auc_score = roc_auc_score(all_video_labels, all_video_probs)
    return auc_score

# --- 실행부: 저장된 베스트 모델 로드 및 평가 ---
for m_name in model_names:
    print(f"\n" + "="*50)
    print(f"📊 [{m_name}] 최종 성능 평가 (Video-level AUC)")
    print("="*50)

    # 1. 베스트 체크포인트 경로 설정
    # CHECKPOINT_DIR가 정의되어 있지 않다면 앞선 셀의 경로와 일치시켜주세요.
    best_model_path = os.path.join(CHECKPOINT_DIR, f"{m_name}_best.pth.tar")

    if os.path.exists(best_model_path):
        # 2. 모델 구조 초기화 및 가중치 로드
        model = timm.create_model(m_name, pretrained=False, num_classes=1).to(device)
        checkpoint = torch.load(best_model_path, map_location=device)

        # 'state_dict' 키가 있는지 확인 후 로드
        if 'state_dict' in checkpoint:
            model.load_state_dict(checkpoint['state_dict'])
        else:
            model.load_state_dict(checkpoint)

        print(f"✅ 베스트 가중치 로드 성공: {best_model_path}")
        if 'epoch' in checkpoint:
            print(f"📅 해당 모델은 Epoch {checkpoint['epoch']}에서 저장된 모델입니다.")

        # 3. AUC 측정 실행
        final_auc = get_video_level_auc(model, val_loader, device)
        print(f"\n🏆 [{m_name}] Final Video-level AUC: {final_auc:.4f}")

        # 4. 결과 요약 리스트 업데이트
        for res in results_summary:
            if res.get('Model') == m_name:
                res['Final_AUC'] = final_auc
    else:
        print(f"❌ 베스트 모델 파일을 찾을 수 없습니다: {best_model_path}")

# 전체 결과 출력
if results_summary:
    print("\n" + "✨" * 5 + " Final Experiment Summary " + "✨" * 5)
    print(pd.DataFrame(results_summary))